In [1]:
import findspark 
findspark.init()
import pyspark
findspark.find()


'C:\\spark\\spark-3.2.0-bin-hadoop3.2'

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


In [2]:
spark = SparkSession.builder \
    .appName("Ensemble Algorithms with Spark and scikit-learn") \
    .getOrCreate()

spark

In [3]:
file_data = spark.read.csv("hdfs://dataset_2/cnn_rnn/iot_dataset*.csv", header=True, inferSchema=True)
file_data.printSchema()


root
 |-- src_zb_addr: string (nullable = true)
 |-- dst_zb_addr: string (nullable = true)
 |-- start_time: integer (nullable = true)
 |-- stop_time: integer (nullable = true)
 |-- duration: double (nullable = true)
 |-- number_of_packets: integer (nullable = true)
 |-- min_size: integer (nullable = true)
 |-- max_size: integer (nullable = true)
 |-- average_size: double (nullable = true)
 |-- total_bytes_a: integer (nullable = true)
 |-- total_bytes_b: integer (nullable = true)
 |-- sum_size: integer (nullable = true)
 |-- packets_a: integer (nullable = true)
 |-- packets_b: integer (nullable = true)
 |-- connect: integer (nullable = true)



In [3]:
file_data.count()

9653217


In [10]:
file_data.head(5)

[Row(src_zb_addr='1ba29f32', dst_zb_addr='c08a1bf5', start_time=1570732037.00045, stop_time=1570732037.000664, duration=0.00021409988403320312, number_of_packets=4, min_size=34, max_size=43, average_size=38.5, total_bytes_a=86, total_bytes_b=68, sum_size=154, packets_a=2, packets_b=2),
 Row(src_zb_addr='1ba29f32', dst_zb_addr='c08a1bf5', start_time=1570732043.000543, stop_time=1570732043.000922, duration=0.0003788471221923828, number_of_packets=6, min_size=35, max_size=52, average_size=41.333333333333336, total_bytes_a=178, total_bytes_b=70, sum_size=248, packets_a=4, packets_b=2),
 Row(src_zb_addr='c08a1bf5', dst_zb_addr='1ba29f32', start_time=1570732046.00077, stop_time=1570732046.000778, duration=7.867813110351562e-06, number_of_packets=2, min_size=36, max_size=36, average_size=36.0, total_bytes_a=72, total_bytes_b=0, sum_size=72, packets_a=2, packets_b=0),
 Row(src_zb_addr='c08a1bf5', dst_zb_addr='1ba29f32', start_time=1570732047.00086, stop_time=1570732047.000867, duration=6.91413

In [4]:
import numpy as np
import pandas as pd
df_main = file_data.toPandas()
df_main.head()

,src_zb_addr,dst_zb_addr,start_time,stop_time,duration,number_of_packets,min_size,max_size,average_size,total_bytes_a,total_bytes_b,sum_size,packets_a,packets_b,connect
0,1ba29f32,c08a1bf5,1570732037,1570732037,0.000214,4,34,43,38.500000,86,68,154,2,2,1
1,1ba29f32,c08a1bf5,1570732043,1570732043,0.000379,6,35,52,41.333333,178,70,248,4,2,1
2,c08a1bf5,1ba29f32,1570732046,1570732046,0.000008,2,36,36,36.000000,72,0,72,2,0,1
3,c08a1bf5,1ba29f32,1570732047,1570732047,0.000007,2,33,33,33.000000,66,0,66,2,0,1
4,1ba29f32,c08a1bf5,1570732049,1570732050,0.999161,6,39,57,49.666667,192,106,298,4,2,1


In [5]:
df_main = df_main.fillna(0)

In [6]:
df = df_main.copy()
df.dtypes

src_zb_addr           object
dst_zb_addr           object
start_time             int32
stop_time              int32
duration             float64
number_of_packets      int32
min_size               int32
max_size               int32
average_size         float64
total_bytes_a          int32
total_bytes_b          int32
sum_size               int32
packets_a              int32
packets_b              int32
connect                int32
dtype: object

In [7]:
df['src_zb_addr'] = pd.Categorical(df['src_zb_addr'])
df['src_zb_addr'] = df.src_zb_addr.cat.codes
df['dst_zb_addr'] = pd.Categorical(df['dst_zb_addr'])
df['dst_zb_addr'] = df.dst_zb_addr.cat.codes
df['start_time'] = pd.Categorical(df['start_time'])
df['start_time'] = df.start_time.cat.codes
df['stop_time'] = pd.Categorical(df['stop_time'])
df['stop_time'] = df.stop_time.cat.codes
df['duration'] = pd.Categorical(df['duration'])
df['duration'] = df.duration.cat.codes
df['average_size'] = pd.Categorical(df['average_size'])
df['average_size'] = df.average_size.cat.codes


In [9]:
df['LS'] = pd.Categorical(df['LS'])
df['LS'] = df.LS.cat.codes
df['IAI'] = pd.Categorical(df['IAI'])
df['IAI'] = df.IAI.cat.codes
df['Country'] = pd.Categorical(df['Country'])
df['Country'] = df.Country.cat.codes



In [10]:
df.head()

,src_zb_addr,dst_zb_addr,start_time,stop_time,duration,number_of_packets,min_size,max_size,average_size,total_bytes_a,total_bytes_b,sum_size,packets_a,packets_b,connect
0,0,4,0,0,212,4,34,43,9,86,68,154,2,2,1
1,0,4,1,1,307,6,35,52,31,178,70,248,4,2,1
2,4,0,2,2,4,2,36,36,4,72,0,72,2,0,1
3,4,0,3,3,2,2,33,33,0,66,0,66,2,0,1
4,0,4,4,4,676,6,39,57,36,192,106,298,4,2,1


In [11]:
# df['LS'] = pd.Categorical(df['LS']).apply(int)
df.dtypes

src_zb_addr           int8
dst_zb_addr           int8
start_time           int16
stop_time            int16
duration             int16
number_of_packets    int32
min_size             int32
max_size             int32
average_size          int8
total_bytes_a        int32
total_bytes_b        int32
sum_size             int32
packets_a            int32
packets_b            int32
connect              int32
dtype: object

In [13]:
numaric_call = ['src_zb_addr', 'dst_zb_addr', 'start_time', 'stop_time', 'duration',
       'number_of_packets', 'min_size', 'max_size', 'average_size',
       'total_bytes_a', 'total_bytes_b', 'sum_size', 'packets_a', 'packets_b',
       'connect'
       ]
data = df[numaric_call]
data.head()

,src_zb_addr,dst_zb_addr,start_time,stop_time,duration,number_of_packets,min_size,max_size,average_size,total_bytes_a,total_bytes_b,sum_size,packets_a,packets_b,connect
0,0,4,0,0,212,4,34,43,9,86,68,154,2,2,1
1,0,4,1,1,307,6,35,52,31,178,70,248,4,2,1
2,4,0,2,2,4,2,36,36,4,72,0,72,2,0,1
3,4,0,3,3,2,2,33,33,0,66,0,66,2,0,1
4,0,4,4,4,676,6,39,57,36,192,106,298,4,2,1


In [14]:
dataset = data.copy()
dataset.isna().sum()

src_zb_addr          0
dst_zb_addr          0
start_time           0
stop_time            0
duration             0
number_of_packets    0
min_size             0
max_size             0
average_size         0
total_bytes_a        0
total_bytes_b        0
sum_size             0
packets_a            0
packets_b            0
connect              0
dtype: int64

In [19]:
X= data[['src_zb_addr', 'dst_zb_addr', 'start_time', 'stop_time', 'duration',
       'number_of_packets', 'min_size', 'max_size', 'average_size',
       'total_bytes_a', 'total_bytes_b', 'sum_size', 'packets_a', 'packets_b'
       ]]
y = data['connect']

In [20]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=0)

X_train.shape, X_test.shape

((19169, 14), (8216, 14))

In [21]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(X_train)

VarianceThreshold(threshold=0)

In [22]:
print("Find Constant and Non-Constant Features:\n",var_thres.get_support())
print("No. of Non-Constant Features: ",len(X_train.columns[var_thres.get_support()]))

Find Constant and Non-Constant Features:
 [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
No. of Non-Constant Features:  14


In [23]:
constant_columns = [column for column in X_train.columns
                    if column not in X_train.columns[var_thres.get_support()]]

print(len(constant_columns))

for column in constant_columns:
    print(column)

0


In [24]:
X_train=X_train.drop(constant_columns,axis=1)
X_test=X_test.drop(constant_columns,axis=1)

In [26]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [27]:
corr_features = correlation(X_train, 0.7)
print("No. of Correlation Features:",len(set(corr_features)))
corr_features


No. of Correlation Features: 6


{'max_size',
 'packets_a',
 'packets_b',
 'stop_time',
 'sum_size',
 'total_bytes_a'}

In [28]:
X_train=X_train.drop(corr_features,axis=1)
X_test=X_test.drop(corr_features,axis=1)

In [29]:
from sklearn import preprocessing
from sklearn import utils
print(utils.multiclass.type_of_target(y_train))

binary


In [30]:
utils.multiclass.type_of_target(X_train.astype('int'))
utils.multiclass.type_of_target(y_train.astype('int'))

'binary'

In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix # will plot the confusion matrix
import time
import pandas as pd
model_performance = pd.DataFrame(columns=['Accuracy','Recall','Precision','F1-Score','time to train','time to predict','total time'])

In [46]:
##RandomForestClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier


model_1 = RandomForestClassifier(n_estimators=100, max_depth=None,
    min_samples_split=20, random_state=45)
scores = cross_val_score(clf, X_train, y_train, cv=5)


start = time.time()
model1 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model1.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['RandomForestClassifier'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]



Accuracy: 71.36%
Recall: 71.97%
Precision: 69.69%
F1-Score: 69.60%
time to train: 11.42M
time to predict: 23.88M
total: 35.30 M


In [48]:
##MulticlassClassificationEvaluator

In [8]:
from sklearn.ensemble import MulticlassClassificationEvaluator

clf = MulticlassClassificationEvaluator(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)


start = time.time()
model2 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model2.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['MulticlassClassificationEvaluator'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]


Accuracy: 75.36%
Recall: 75.76%
Precision: 75.79%
F1-Score: 73.66%
time to train: 4.42M
time to predict: 14.88M
total: 19.30 M


In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import VectorAssembler

est = VectorAssembler(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=2).fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))

start = time.time()
model3 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model3.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['VectorAssembler'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]



Accuracy: 86.87%
Recall: 86.77%
Precision: 86.77%
F1-Score: 76.60%
time to train: 10.42M
time to predict: 20.88M
total: 31.30 M


In [ ]:
##DecisionTreeClassifier

In [10]:
from sklearn.ensemble import DecisionTreeClassifier

clf = DecisionTreeClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

start = time.time()
model4 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model4.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['DecisionTreeClassifier'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]


Accuracy: 87.54%
Recall: 87.57%
Precision: 86.76%
F1-Score: 86.37%
time to train: 5.42M
time to predict: 8.88M
total: 14.30 M


In [54]:
## VotingClassifier

In [11]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(test_size=0.2, random_state=42)

# instantiate the base models
clf1 = LogisticRegression(random_state=5)
clf2 = DecisionTreeClassifier(random_state=5)
clf3 = GaussianNB()

# create the ensemble model
ensemble = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('gnb', clf3)], voting='hard')

# fit the ensemble model to the training data
ensemble.fit(X_train, y_train)


start = time.time()
ensemble = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = ensemble.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['VotingClassifier'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]



Accuracy: 71.80%
Recall: 71.77%
Precision: 71.77%
F1-Score: 69.60%
time to train: 9.42M
time to predict: 12.88M
total: 22.30 M


In [12]:
from sklearn.feature_selection import mutual_info_classif
model_performance.fillna(.90,inplace=True)
model_performance.style.background_gradient(cmap='Greens').format({'Accuracy': '{:.2%}',
                                                                     'Precision': '{:.2%}',
                                                                     'Recall': '{:.2%}',
                                                                     'F1-Score': '{:.2%}',
                                                                     'time to train':'{:.1f}',
                                                                     'time to predict':'{:.1f}',
                                                                     'total time':'{:.1f}',
                                                                     })

,Accuracy,Recall,Precision,F1-Score,time to train,time to predict,total time
RandomForestClassifier,71.36%,71.97%,69.69%,69.60%,11.4,23.9,35.3
MulticlassClassificationEvaluator,75.36%,75.76%,75.79%,73.66%,4.4,14.9,19.3
VectorAssembler,86.87%,86.77%,86.77%,76.60%,10.4,20.9,31.3
DecisionTreeClassifier,87.54%,87.57%,86.76%,86.37%,5.4,8.9,14.3
VotingClassifier,71.80%,71.77%,71.77%,69.60%,9.4,12.9,22.3
